In [0]:
%fs ls /mnt/Anac

In [0]:
%fs ls dbfs:/mnt/Anac/Silver/

In [0]:
df = spark.read.parquet("dbfs:/mnt/Anac/Silver/anac_silver.parquet/")
df.limit(100).display()

In [0]:
#analisando colunas
print(df.columns)

In [0]:
#adicionando uma nova coluna somando totas as lesões
colunas_a_somar = ['Lesoes_Desconhecidas_Passageiros', 'Lesoes_Desconhecidas_Terceiros', 'Lesoes_Desconhecidas_Tripulantes', 'Lesoes_Fatais_Passageiros', 'Lesoes_Fatais_Terceiros', 'Lesoes_Fatais_Tripulantes', 'Lesoes_Graves_Passageiros', 'Lesoes_Graves_Terceiros', 'Lesoes_Graves_Tripulantes', 'Lesoes_Leves_Passageiros', 'Lesoes_Leves_Terceiros', 'Lesoes_Leves_Tripulantes']

df = df.withColumn("Total_Lesoes", sum(df[somartudo] for somartudo in colunas_a_somar))
df.limit(100).display()


In [0]:
print(df.columns)

In [0]:
#renomear colunas para facilitar
df = df\
    .withColumnRenamed("Aerodromo_de_Destino", "Destino")\
    .withColumnRenamed("Aerodromo_de_Origem", "Origem")\
    .withColumnRenamed("Categoria_da_Aeronave", "Categoria")\
    .withColumnRenamed("Classificacao_da_Ocorrencia", "Classificacao")\
    .withColumnRenamed("Danos_a_Aeronave", "Danos")\
    .withColumnRenamed("Data_da_Ocorrencia", "Data")\
    .withColumnRenamed("UF", "Estado")

In [0]:
#excluir estados que tenham a classificação [Indeterminado, Sem Registro, Exterior]
filtro = ['Indeterminado', 'Sem Registro', 'Exterior']
df = df.filter(~df['Estado'].isin(filtro))
display(df)

In [0]:
#inserir colunas com nome de atualizado
from pyspark.sql.functions import current_timestamp, date_format, from_utc_timestamp

df = df\
    .withColumn("Atualizacao",
        date_format(from_utc_timestamp(current_timestamp(), "America/Sao_Paulo"), "dd/MM/yyyy HH:mm:ss"))

df.limit(100).display()

In [0]:
# salvar na camada Gold

df.write\
    .mode("overwrite")\
    .format("parquet")\
    .partitionBy("Estado")\
    .save("dbfs:/mnt/Anac/Gold/anac_gold.parquet/")

In [0]:
%fs ls /mnt/Anac/Gold

In [0]:
df = spark.read.parquet("dbfs:/mnt/Anac/Gold/anac_gold.parquet/")
df.limit(100).display()